In [13]:
'''
Nisarg Thakkar

Scrape the Dell laptop community, and find out how many posts are there for Inspiron, XPS and Latitude

Scrape all discussion from the first 5 pages for the Inspiron sub-community. Find discussion title, author of the question,
time of the question, author of the latest post, time of the latest post, number of kudos, number of replies, 
number of views, whether the question is solved or not (indicated using green checkmark). 
Then save these information to a csv file.

https://www.dell.com/community/Laptops/ct-p/Laptops
'''

#import libraries

from urllib.request import urlopen
from bs4 import BeautifulSoup
import os, csv
import pandas as pd

#Specify url, get the webpage and return it to varibale page
page = urlopen("https://www.dell.com/community/Laptops/ct-p/Laptops")
#print(page)

#parse the html page and store in variable soup
soup = BeautifulSoup(page, 'html.parser')


In [2]:
# Scrape the Dell laptop community, and find out how many posts are there for Inspiron, XPS and Latitude

d_name_post = {}
j = [0, 1, 3]

for i in j:    

    names = soup.find_all('div', attrs = {"cat-card-title"})
    names = names[i].text

    i_post = soup.find_all('span', attrs = {'class': "cat-card-posts"})
    i_post = i_post[i].text.strip()
    
    d_name_post[names] = i_post

print(d_name_post)

{'Inspiron': '12,271 posts', 'XPS': '5,505 posts', 'Latitude': '3,725 posts'}


In [27]:
''' Scrape all discussion from the first 5 pages for the Inspiron sub-community. Find discussion title, 
 author of the question, time of the question, author of the latest post, time of the latest post, number of kudos, number of replies, 
number of views, whether the question is solved or not (indicated using green checkmark). 
Then save these information to a csv file.
'''

dic = {}
#Specify url, get the webpage and return it to varibale page
for pg_number in range(1, 6): 
    
   
    inspiron_page = urlopen("https://www.dell.com/community/Inspiron/bd-p/Inspiron/page/{}".format(pg_number))
    #print(page)

    #parse the html page and store in variable soup
    soup = BeautifulSoup(inspiron_page, 'html.parser')
    
    import re
    
    whole_post = soup.find_all('div', attrs = {"class" : 'lia-component-messages-column-thread-info'})
    #Getting a whole post row
    
    solved_container = soup.find_all('td', attrs = {'class' : 'triangletop lia-data-cell-secondary lia-data-cell-icon'})
    #getting a container tag to find out if post is solved or not
    
   
    #going through each post in that page
    for z in range(len(whole_post)):
        st = ((whole_post[z].text.strip()).replace("\n","").replace("\t","").replace("  ",""))
        #print(st)
        date = soup.find_all('span', attrs = {'class' : 'DateTime lia-component-common-widget-date'})
        date = date[z].text.strip()

        views = soup.find_all('div', attrs = {'class' : 'lia-component-messages-column-topic-views-count'})
        views = views[z].text.strip()[0:5] #getting views on that post

        by_pos = st.find("by")
        title = st[:by_pos]
        on_pos = st.find(" on ")
        author = st[by_pos + 3 : on_pos]
        if len(author) >= 15:
            author = 'None'

        first_on_pos = st.find(" on ")
        last_on_pos = st.rfind(" on ")
        #print(first_on_pos, last_on_pos)
        latest_comment_date = 'None' #Intialing it to 'None'
        latest_auth = 'None'

        if first_on_pos != last_on_pos:#finding if there're multiple dates to find out if someone commented on it or not        

            last_by_pos = st.rfind(" by ")
            replies_pos = st.rfind("Reply")

            latest_comment_date = st[last_on_pos + 4 : last_by_pos]
            #print('Latest comment date :', latest_comment_date)

            if last_by_pos != by_pos: 
                latest_auth = st[last_by_pos + 4 : replies_pos - 1]
                #print('Latest comment made by : ', latest_auth)


        kudos = soup.find_all('td', attrs = {'class' : 'cKudosCountColumn lia-data-cell-secondary lia-data-cell-integer'})
        #print(kudos)
        kudos = kudos[z].text.strip()

        message_count = soup.find_all('td', attrs = {'class' : 'cRepliesCountColumn lia-data-cell-secondary lia-data-cell-integer'})
        message_count = message_count[2].text.strip()

        row = soup.find_all('tr', {'class' :re.compile('lia-js-data-messageUid-\d+')})
        #print(len(row))
        solved = row[z].find('td', attrs = {'aria-label' : 'This thread is solved'})
   
        if solved == None: #if solved tag exists, post is solved, if None return it does not
            solved ="Not Solved"
        else:
            solved = 'solved'
            
        dic[title] = [author, date, latest_auth, latest_comment_date, kudos, message_count, views, solved]
#print(dic)

names = ['author', 'date', 'latest_author', 'latest_comment_date', 'kudos', 'message_count', 'views', 'solved'] 


(pd.DataFrame.from_dict(data=dic, orient='index')
.to_csv('Nisarg_web_scrapping.csv', header= names ))
    